In [ ]:
%load_ext autoreload
%autoreload 2

import os
import uproot
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
from matplotlib.legend_handler import HandlerLine2D, HandlerTuple
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats
import datetime as dt
import scipy.optimize
import landau
from scipy.interpolate import CubicSpline, interp2d
from scipy.integrate import quad
from scipy.special import erf
from numpy import linalg

import importlib

# local imports
from lib.constants import *

In [ ]:
dosave = True
savedata = False

plt.rcParams.update({'font.size': 14})
plotqual = "runa_"
savedir = "./plots_6_28_23/%s" % plotqual
plottitle = "Run A"
tpcnames = ["EE", "EW", "WE", "WW"]

# datadir = "/icarus/data/users/gputnam/drift-normalization/"
datadir = "/icarus/data/users/gputnam/calib-data/outputs/"

In [ ]:
# Reference df
mpv_df0 = pd.read_hdf("/icarus/data/users/gputnam/calib-data/dfs/mpvs_muon_Run1.df")

In [ ]:
mpv_df = pd.read_hdf("/icarus/data/users/gputnam/calib-data/dfs/mpvs_muon_RunA.df")

In [ ]:
mpv_df

In [ ]:
pitches = mpv_df.pitch.unique()
drifts = mpv_df.tdrift.unique()
rrs = np.array(list(reversed(mpv_df.rr.unique())))
names = [
    "TPC_EE",
    "TPC_EW",
    "TPC_WE",
    "TPC_WW"
]
# names = ["MC"]

In [ ]:
rrmin = 200

In [ ]:
# dQdx binning
bins = np.linspace(200, 4000, 38*4+1)
bin_centers = (bins[1:] + bins[:-1]) / 2.

def landau_gaus(X, *p):
    mpv, eta, sigma, A = p
    sigma = np.minimum(sigma, 100*eta)
    return landau.landau.gauss_landau(X, mpv, eta, sigma, A)

In [ ]:
ifig = 0

for tdrift in drifts:
    plt.figure(ifig)

    for n in names:
        when = (mpv_df.tdrift == tdrift) & (mpv_df[n]) & (mpv_df.rr > rrmin)
        
        plt.plot(rrs[rrs > rrmin], mpv_df.phi[when], label=n)

    plt.ylabel("$\\langle \\phi \\rangle$")
    plt.xlabel("Residual Range [cm]")

    ifig += 1

In [ ]:
ifig = 0

for tdrift in drifts:
    plt.figure(ifig)

    for n in names:
        when = (mpv_df.tdrift == tdrift) & (mpv_df[n]) & (mpv_df.rr > rrmin)
        
        plt.errorbar(rrs[rrs > rrmin], mpv_df.mpv[when], mpv_df.mpv_err[when], 
                     label=n[-2:], linestyle="none", marker=".")

    plt.ylabel("dQ/dx [ADDC/cm]")
    plt.xlabel("Residual Range [cm]")
    plt.title("%s $t_\\mathrm{drift}: %i\\mu$s" % (plottitle, tdrift))
    ifig += 1
    plt.ylim([plt.ylim()[0]-7, plt.ylim()[1]])
    plt.tight_layout()
    h, l = plt.gca().get_legend_handles_labels()
    ph = [plt.plot([],marker="", ls="")[0]]
    handles = ph + h
    labels = ["TPC: "] + l
    leg = plt.legend(handles, labels, ncol=5, 
                   loc='lower center', columnspacing=0.8, handletextpad=0.4)
    
    if dosave:
        plt.savefig(savedir + "chargeratios_drift%i.pdf" % int(tdrift))

In [ ]:
# Compute the gain ratio to normalize against EW
name0 = "TPC_EW"

In [ ]:
ratios = []
for n in names:
    mpvdf0 = mpv_df0[mpv_df0[name0] & (mpv_df0.rr > rrmin)].sort_values(["tdrift", "rr"])
    mpvdfn = mpv_df[mpv_df[n] & (mpv_df.rr > rrmin)].sort_values(["tdrift", "rr"])
    
    ratio = np.mean(mpvdf0.mpv.values / mpvdfn.mpv.values)
    ratios.append(ratio)
    print(n, ratio)

In [ ]:
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

In [ ]:
for i, n in enumerate(names):
    means = []
    errs = []
    for tdrift in drifts:
        when = (mpv_df.tdrift == tdrift) & (mpv_df[n]) & (mpv_df.rr > rrmin)
        when0 = (mpv_df0.tdrift == tdrift) & (mpv_df0[name0]) & (mpv_df0.rr > rrmin)
        
        mpv_mean = np.mean(mpv_df.mpv[when]) / np.mean(mpv_df0.mpv[when0])
        mpv_err = np.sqrt(1/np.sum(1/mpv_df.mpv_err[when]**2))  / np.mean(mpv_df0.mpv[when0])
        means.append(mpv_mean)    
        errs.append(mpv_err)
        
    plt.errorbar(drifts, means, yerr=errs, label=n[-2:],
                linestyle="none", marker=".", color=colors[i])

    plt.axhline([1/ratios[i]], color=colors[i])
    
plt.ylim([plt.ylim()[0]-0.01, plt.ylim()[1]])

h, l = plt.gca().get_legend_handles_labels()
ph = [plt.plot([],marker="", ls="")[0]]
handles = ph + h
labels = ["TPC: "] + l
leg = plt.legend(handles, labels, ncol=5, 
               loc='lower center', columnspacing=0.8, handletextpad=0.4)

plt.title(plottitle)    
plt.xlabel("Drift Time [$\\mu$s]")
plt.ylabel("dQ/dx Relative Scale")

if dosave:
    plt.savefig(savedir + "tpc_equalization.pdf")

In [ ]:
if savedata:
    with open(datadir + "tpc_ratio_RunA.txt", "w") as f:
        for n, r in zip(names, ratios):
            f.write("%s %.4f\n" % (n[-2:], r))